统计数据长度

In [2]:
def analyze_txt_file(file_path):
    lengths = []
    
    # 读取文件内容并计算每行的长度
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_length = len(line.strip())  # 去除首尾空格
            if line_length > 0:  # 过滤掉空行
                lengths.append(line_length)
    
    # 计算平均长度、最小长度和最大长度
    if lengths:
        avg_length = sum(lengths) / len(lengths)
        min_length = min(lengths)
        max_length = max(lengths)
        max_line_number = lengths.index(max_length) + 1  # 行数从1开始
        
        print(f"第一行的长度为: {lengths[0]}")
        print(f"总共有 {len(lengths)} 行文本")
        print(f"平均长度为: {avg_length:.2f}")
        print(f"最短行的长度为: {min_length}")
        print(f"最长行的长度为: {max_length}，位于第 {max_line_number} 行")
    else:
        print("没有有效的文本行")

# 调用函数并传入你的文件路径
file_path = '/home/extra1T/kangh/app/kh/corpus.txt'
analyze_txt_file(file_path)


第一行的长度为: 181
总共有 50000 行文本
平均长度为: 900.67
最短行的长度为: 8
最长行的长度为: 46909，位于第 37969 行


BM25实现

In [ ]:
import jieba
from rank_bm25 import BM25Okapi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import torch
# 加载并处理文本
def load_and_process_txt(file_path):
    loader = TextLoader(file_path, encoding='utf8')
    documents = loader.load()
    
    # 分割文本，保持上下文一致性
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

# 进行分词处理
def process_for_bm25(docs):
    # 对每个文档进行分词，得到一个二维数组
    tokenized_docs = [jieba.lcut(doc.page_content) for doc in docs]
    return tokenized_docs

# 文档路径
file_path = '/home/extra1T/kangh/app/kh/corpus.txt'

# 加载并分割文本
docs = load_and_process_txt(file_path)
# print("dsad:",len(docs))
# print("dsads:",docs[0].page_content)
# 对文档进行分词，准备给 BM25 使用
tokenized_docs = process_for_bm25(docs)

# print("dsad:",tokenized_docs[0])

# 初始化 BM25 模型
bm25 = BM25Okapi(tokenized_docs)

# 输入查询，查询的内容需要分词
query = "第8届南方新丝路模特大赛的招募活动是在哪里举行的？"
query_tokens = jieba.lcut(query)

# 使用 BM25 进行检索，得到每个文档的相关性分数
doc_scores = bm25.get_scores(query_tokens)
print(len(doc_scores))

# 设置要返回的 topk 个文档
topk = 3  # 例如返回前3个相似答案

# 获取分数最高的 topk 个文档索引
topk_indices = doc_scores.argsort()[-topk:][::-1]

# 获取分数最高的 topk 个文档内容
topk_matches = [docs[idx].page_content for idx in topk_indices]

# 输出 topk 个最相关的文档内容
print("最相关的文档内容：")
for i, match in enumerate(topk_matches, 1):
    print(f"Top {i}:\n{match}\n")
